In [1]:
import tensorflow as tf
import os

class MyFashionMnist(object):
  def train(self):
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()

    print("Training...")
    
    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 
    
    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        
        PRIVATE_REGISTRY = 'registry.kube-system.svc.cluster.local:30000'
        
        # fairing.config.set_preprocessor(
        #     'notebook', 
        #     command = ['python3'],  # default: python
        # )
        
        fairing.config.set_builder(
            'append',
            # base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
            base_image = 'tensorflow/tensorflow:2.0.3-gpu-py3',
            registry = PRIVATE_REGISTRY,
            image_name='my-04-notebook-single-file-fairing-job', 
            push=True
        )
        
        fairing.config.set_deployer(
            'job',
            namespace='myspace',
            pod_spec_mutators=[
                k8s_utils.get_resource_mutator(cpu=1, memory=5)]
        )
        
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 210103 17:22:51 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7f70d4b71b70>
[I 210103 17:22:51 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f715f8b27b8>
[I 210103 17:22:51 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f70daecc550>
[W 210103 17:22:51 append:50] Building image using Append builder...
[I 210103 17:22:51 base:107] Creating docker context: /tmp/fairing_context_tzpcinxn
[I 210103 17:22:51 converted_notebook:127] Converting 04-notebook-single-file-fairing.ipynb to 04-notebook-single-file-fairing.py
[I 210103 17:22:51 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:2.0.3-gpu-py3'
[W 210103 17:22:53 append:54] Image successfully built in 2.116938926046714s.
[W 210103 17:22:53 append:94] Pushing image registry.kube-system.svc.cluster.local:30000/my-04-notebook-single-file-fairing-j

11493376/11490434 [==============================] - 0s 0us/step
2021-01-03 17:23:21.760340: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-01-03 17:23:21.760427: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (-1)
2021-01-03 17:23:21.760464: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fairing-job-q5s9g-8sw48): /proc/driver/nvidia/version does not exist
2021-01-03 17:23:21.760785: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-01-03 17:23:21.769844: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-01-03 17:23:21.770445: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x48a4650 executing computations on platform Host. Devices:
2021-01-03 17:23:

[W 210103 17:23:57 job:173] Cleaning up job fairing-job-q5s9g...
